<a href="https://colab.research.google.com/github/mathu3004/Pearl_Path/blob/Chatbot/Chatbot_code_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gemini  # Assuming gemini is the API client library

# Initialize the Gemini API client
gemini_client = gemini.Client(api_key="YOUR_GEMINI_API_KEY")

# Function to greet the user
def greet_user():
    return "Hello! Welcome to our trip itinerary web application. How can I assist you today?"

# Function to analyze user input using Gemini API
def analyze_input(user_input):
    # Use Gemini API to detect the intent
    response = gemini_client.detect_intent(user_input)

    # Map detected intents to functionalities
    intent = response.get("intent")
    if intent == "emergency_assistance":
        return "emergency_assistance"
    elif intent == "information_hub":
        return "information_hub"
    elif intent == "user_provider_connection":
        return "user_provider_connection"
    elif intent == "recommendations":
        return "recommendations"
    else:
        return "unknown"

# Main function to handle user interaction
def main():
    # Greet the user
    print(greet_user())

    # Simulate user input (for demonstration purposes)
    user_input = input("You: ")

    # Analyze the user input using Gemini API
    functionality = analyze_input(user_input)

    # Print the identified functionality (for demonstration purposes)
    if functionality == "emergency_assistance":
        print("Directing to Emergency Assistance functionality.")
    elif functionality == "information_hub":
        print("Directing to Information Hub functionality.")
    elif functionality == "user_provider_connection":
        print("Directing to User Provider Connection functionality.")
    elif functionality == "recommendations":
        print("Directing to Recommendations functionality.")
    else:
        print("Sorry, I didn't understand that. Can you please rephrase?")

# Run the main function
if __name__ == "__main__":
    main()
